In [303]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.arima_model import _arma_predict_out_of_sample
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller

### Importing & Building Raw Data Set

In [304]:
file_lst = os.listdir('./raw_data_nfl_QB/')
for idx,f in enumerate(file_lst):
    if idx == 0:
        data = pd.read_csv('./raw_data_nfl_QB/{}'.format(f))
    else:
        temp_df = pd.read_csv('./raw_data_nfl_QB/{}'.format(f))
        data = pd.concat([data,temp_df],axis = 0)

In [305]:
data.reset_index(inplace=True)

### Renaming

In [306]:
data['Rank_QBR'] = data['Rk']
data['Sack'] = data['Sk']
data['Completions'] = data['Cmp']
data['Attempts'] = data['Att']
data['Passing_Yards'] = data['Yds']
data['Sack_Yards'] = data['Yds.1']
data['QBR'] = data['Rate']
data['Yds_Per_Attempt'] =  data['Y/A']
data['Adj_Yd_Per_Attempt'] = data['AY/A']

### Feature Engineering

Assuming basic scoring for calculation of fantasy points column

    25 yards = 1 pt
    1 TD = 6 pts
    1 Int = -2 pts

In [307]:
data['Year'] = data['Date'].str.split('-', 1, expand=True)[0]
data['Year'] = data['Year'].astype(str).astype(int)
data['Month'] = data['Date'].str.split('-', 2, expand=True)[1]
data['Month'] = data['Month'].astype(str).astype(int)
data['Name'] = data['Unnamed: 1'].str.split('\\', 1, expand=True)[0]
data['Home_Away'] = (data['Unnamed: 6']=='@').map({False:'Home', True:'Away'})
data['Home_Team'] = (data['Home_Away']=='Home').map({True:1, False:0})
data['Fantasy_Points'] = (data['Passing_Yards']/25)+(data['TD']*6)-(data['Int']*2)

In [308]:
QB_ID = []
for a, b in zip(data['Year'], data['Name']):
    Qid = '{}_{}'.format(a,b)
    QB_ID.append(Qid)
    
data['QB_ID'] = QB_ID

### Dropping Irrelevant & Redundant Columns

In [309]:
## Dropping Columns that have been renamed or are irrelevant('Lg')
drop_cols = ['Rk', 'Sk', 'Cmp', 'Yds', 'Lg', 'Att', 'Yds', 'Yds.1',
             'Rate', 'Y/A', 'AY/A', 'index', 'Unnamed: 1', 'Unnamed: 6']
data.drop(drop_cols, axis=1, inplace=True)

### Trimming Dataframe to Passing Attempts >= 10

In [310]:
## Dropping Attempts Below 5
data = data.loc[data['Attempts']>=10]

In [311]:
data.columns

Index(['Age', 'Date', 'Tm', 'Opp', 'Result', 'G#', 'Week', 'Day', 'Cmp%', 'TD',
       'Int', 'Rank_QBR', 'Sack', 'Completions', 'Attempts', 'Passing_Yards',
       'Sack_Yards', 'QBR', 'Yds_Per_Attempt', 'Adj_Yd_Per_Attempt', 'Year',
       'Month', 'Name', 'Home_Away', 'Home_Team', 'Fantasy_Points', 'QB_ID'],
      dtype='object')

### Check for Stationarity

In [312]:
results_Yds = adfuller(data['Passing_Yards'].values)
results_TD = adfuller(data['TD'].values)
results_Int = adfuller(data['Int'].values)

In [313]:
print('Yards Dickey Fuller Test')
print(results_Yds[0], 'T-Statistic')
print(results_Yds[1], 'P_value')
print(results_Yds[4], 'Critical Values')
print('TD Dickey Fuller Test')
print(results_TD[0], 'T-Statistic')
print(results_TD[1], 'P_value')
print(results_TD[4], 'Critical Values')
print('Int Dickey Fuller Test')
print(results_Int[0], 'T-Statistic')
print(results_Int[1], 'P_value')
print(results_Int[4], 'Critical Values')

Yards Dickey Fuller Test
-9.9578607912 T-Statistic
2.41723576211e-17 P_value
{'1%': -3.4329527780962255, '5%': -2.8626898965523724, '10%': -2.567382133955709} Critical Values
TD Dickey Fuller Test
-11.2758637549 T-Statistic
1.49719912924e-20 P_value
{'1%': -3.4329527780962255, '5%': -2.8626898965523724, '10%': -2.567382133955709} Critical Values
Int Dickey Fuller Test
-11.7313487594 T-Statistic
1.33935739303e-21 P_value
{'1%': -3.4329527780962255, '5%': -2.8626898965523724, '10%': -2.567382133955709} Critical Values


In [15]:
### shows variables are stationary

In [314]:
## Break Down By Week
Week1_df = data.loc[data['Week']==1]
Week2_df = data.loc[data['Week']==2]
Week3_df = data.loc[data['Week']==3]
Week4_df = data.loc[data['Week']==4]
Week5_df = data.loc[data['Week']==5]
Week6_df = data.loc[data['Week']==6]
Week7_df = data.loc[data['Week']==7]
Week8_df = data.loc[data['Week']==8]
Week9_df = data.loc[data['Week']==9]
Week10_df = data.loc[data['Week']==10]
Week11_df = data.loc[data['Week']==11]
Week12_df = data.loc[data['Week']==12]
Week13_df = data.loc[data['Week']==13]
Week14_df = data.loc[data['Week']==14]
Week15_df = data.loc[data['Week']==15]
Week16_df = data.loc[data['Week']==16]
Week17_df = data.loc[data['Week']==17]

In [315]:
## Break Down By Season
df_2017 = data.loc[data['Year']==2017]
df_2016 = data.loc[data['Year']==2016]
df_2015 = data.loc[data['Year']==2015]
df_2014 = data.loc[data['Year']==2014]
df_2013 = data.loc[data['Year']==2013]

In [316]:
X = data[['Cmp%', 'TD', 'Int', 'Sack', 'Completions', 'Attempts','Sack_Yards', 'QBR']]
y = data['Passing_Yards']
lr_yards = LinearRegression()
lr_yards.fit(X,y)
predict_yards = lr_yards.predict(X)
residuals_yards = predict_yards - y.values
ar_yards = ARMA(residuals_yards, (2,1)).fit()

In [317]:
def yard_projections(mstr_df,qb_id,week,lr_yards,ar_yards):
    '''
    mstr_df: Pandas DF; master df with all qbs in all years and all weeks
    qb_id: string; the id of the QB you want to make a prediction
    week: int, predicting week
    lr_yards: linear regression model for yards
    ar_yards: ARMA model object for yard residuals
    
    '''
    ycol = ['Passing_Yards'] ## Target Variable
    Xcols = ['Cmp%', 'TD', 'Int', 'Sack', 'Completions', 'Attempts','Sack_Yards', 'QBR'] ## predictors
    ## create temp df for each QB by Year and week
    ## then reset index
    temp_df = mstr_df[mstr_df['QB_ID'] == qb_id]
    temp_df = temp_df.sort_values('G#')
    temp_df.reset_index(inplace=True)
    X = temp_df[Xcols]
    y = temp_df[ycol]
    ## grabbing the weeks before the week specified
    w = week-1
    prev_weeks = temp_df[:w][Xcols]
    y_pred = lr_yards.predict(prev_weeks)
    resid = np.array(y[ycol][:w]).flatten()-y_pred
    dates = temp_df.index
    params = ar_yards.params
    residuals = ar_yards.resid
    p = ar_yards.k_ar
    q = ar_yards.k_ma
    k_exog = ar_yards.k_exog
    k_trend = ar_yards.k_trend
    steps = 1
    
    # this is the error
    
#     print("Residuals:")
#     print(residuals)
#     print("Params:")
#     print(params)
#     print("Steps:")
#     print(steps)
#     print("p,q:")
#     print(p)
#     print('/n')
#     print(q
#     print("Resid:")
#     resid
#     print("w: {}".format(w))
#     return
    
    oos_predictions_yards_resid = _arma_predict_out_of_sample(params, steps, residuals, 
                                    p, q, k_trend, k_exog, 
                                    endog=resid, exog=None, start=w+1)
    
    pred_yds = oos_predictions_yards_resid + (y_pred.sum()/week)
    return pred_yds
#     start = dates[w]
#     end = dates[w]
#     residual_prediction = ar_yards.predict(start=start, end=end, dynamic=True)
#     pred_yds = residual_prediction + (y_pred.sum()/week)
#     return pred_yds

In [318]:
X = data[['Cmp%', 'Passing_Yards', 'Int', 'Sack', 'Completions', 'Attempts','Sack_Yards', 'QBR']]
y = data['TD']
lr_TD = LinearRegression()
lr_TD.fit(X,y)
predict_TD = lr_TD.predict(X)
residuals_TD = predict_TD - y.values
ar_TD = ARMA(residuals_TD, (2,1)).fit()

In [319]:
def TD_projections(mstr_df,qb_id,week,lr_TD,ar_TD):
    '''
    mstr_df: Pandas DF; master df with all qbs in all years and all weeks
    qb_id: string; the id of the qq you want to make a prediction
    week: int, predicting week
    lr_yards: linear regression model for yards
    ar_yards: ARMA model object for yard residuals
    
    '''
    ycol = ['TD'] ## Target Variable
    Xcols = ['Cmp%', 'Passing_Yards', 'Int', 'Sack', 'Completions', 'Attempts','Sack_Yards', 'QBR'] ## predictors
    ## create temp df for each QB by Year and week
    ## then reset index
    temp_df = mstr_df[mstr_df['QB_ID'] == qb_id]
    temp_df = temp_df.sort_values('G#')
    temp_df.reset_index(inplace=True)
    X = temp_df[Xcols]
    y = temp_df[ycol]
    ## grabbing the weeks before the week specified
    w = week-1
    prev_weeks = temp_df[:w][Xcols]
    y_pred = lr_TD.predict(prev_weeks)
    resid = np.array(y[ycol][:w]).flatten()-y_pred
    dates = temp_df.index
    params = ar_TD.params
    residuals = ar_TD.resid
    p = ar_TD.k_ar
    q = ar_TD.k_ma
    k_exog = ar_TD.k_exog
    k_trend = ar_TD.k_trend
    steps = 1
    
    oos_predictions_TD_resid = _arma_predict_out_of_sample(params, steps, residuals, 
                                    p, q, k_trend, k_exog, 
                                    endog=resid, exog=None, start=w+1)
    
    pred_TD = oos_predictions_TD_resid + (y_pred.sum()/week)
    return pred_TD
    
#     start = dates[w]
#     end = dates[w]
#     residual_prediction = ar_TD.predict(start=start, end=end, dynamic=True)
#     pred_TDs = residual_prediction + (y_pred.sum()/week)
#     return pred_TDs

In [320]:
X = data[['Cmp%', 'Passing_Yards', 'TD', 'Sack', 'Completions', 'Attempts','Sack_Yards', 'QBR']]
y = data['Int']
lr_Int = LinearRegression()
lr_Int.fit(X,y)
predict_Int = lr_Int.predict(X)
residuals_Int = predict_Int - y.values
ar_Int = ARMA(residuals_Int, (2,1)).fit()

In [321]:
def Int_projections(mstr_df,qb_id,week,lr_Int,ar_Int):
    '''
    mstr_df: Pandas DF; master df with all qbs in all years and all weeks
    qb_id: string; the id of the qq you want to make a prediction
    week: int, predicting week
    lr_yards: linear regression model for yards
    ar_yards: ARMA model object for yard residuals
    
    '''

    ycol = ['Int'] ## Target Variable
    Xcols = ['Cmp%', 'Passing_Yards', 'TD', 'Sack', 'Completions', 'Attempts','Sack_Yards', 'QBR'] ## predictors
    ## create temp df for each QB by Year and week
    ## then reset index
    temp_df = mstr_df[mstr_df['QB_ID'] == qb_id]
    temp_df = temp_df.sort_values('G#')
    temp_df.reset_index(inplace=True)
    X = temp_df[Xcols]
    y = temp_df[ycol]
    ## grabbing the weeks before the week specified
    w = week-1
    prev_weeks = temp_df[:w][Xcols]
    y_pred = lr_Int.predict(prev_weeks)
    resid = np.array(y[ycol][:w]).flatten()-y_pred
    dates = temp_df.index
    params = ar_Int.params
    residuals = ar_Int.resid
    p = ar_Int.k_ar
    q = ar_Int.k_ma
    k_exog = ar_Int.k_exog
    k_trend = ar_Int.k_trend
    steps = 1

    oos_predictions_Int_resid = _arma_predict_out_of_sample(params, steps, residuals, 
                                    p, q, k_trend, k_exog, 
                                    endog=resid, exog=None, start=w+1)

    pred_Int = oos_predictions_Int_resid + (y_pred.sum()/week)
    return pred_Int
#         print("the try ran")
#     except:
#         ycol = ['Int'] ## Target Variable
#         print("the except ran")
#         return ycol
#         Xcols = ['Cmp%', 'Passing_Yards', 'TD', 'Sack', 'Completions', 'Attempts','Sack_Yards', 'QBR'] ## predictors
#         temp_df = mstr_df[mstr_df['QB_ID'] == qb_id]
#         return temp_df

#     start = dates[w]
#     end = dates[w]
#     residual_prediction = ar_Int.predict(start=start, end=end, dynamic=True)
#     pred_Ints = residual_prediction + (y_pred.sum()/week)
#     return pred_Ints

In [271]:
Int_projections(data, '2017_Tom Savage', 15, lr_Int,ar_Int)

the try ran


array([ 0.44987674])

## Predict week 4 for Peyton

In [322]:
def Fantasy_Points_Calculator(QB_ID, week):
    Fantasy_point_prediction = (((yard_projections(data, QB_ID, week, lr_yards, ar_yards))/25)
    +((TD_projections(data, QB_ID, week, lr_TD, ar_TD))*6)
    -((Int_projections(data, QB_ID, week, lr_Int, ar_Int))*2))
    return Fantasy_point_prediction

In [143]:
params = ar_yards.params
residuals = ar_yards.resid
p = ar_yards.k_ar
q = ar_yards.k_ma
k_exog = ar_yards.k_exog
k_trend = ar_yards.k_trend
steps = 1

oos_predictions_yards_resid = _arma_predict_out_of_sample(params, steps, residuals_yards, 
                                p, q, k_trend, k_exog, 
                                endog=residuals, exog=None, start=15)

In [144]:
params = ar_TD.params
residuals = ar_TD.resid
p = ar_TD.k_ar
q = ar_TD.k_ma
k_exog = ar_TD.k_exog
k_trend = ar_TD.k_trend
steps = 1

oos_predictions_TD_resid = _arma_predict_out_of_sample(params, steps, residuals_TD, 
                                p, q, k_trend, k_exog, 
                                endog=residuals, exog=None, start=15)

In [145]:
params = ar_yards.params
residuals = ar_yards.resid
p = ar_yards.k_ar
q = ar_yards.k_ma
k_exog = ar_yards.k_exog
k_trend = ar_yards.k_trend
steps = 1

oos_predictions_Int_resid = _arma_predict_out_of_sample(params, steps, residuals_Int, 
                                p, q, k_trend, k_exog, 
                                endog=residuals, exog=None, start=15)

In [96]:
pqbs = df_2017['QB_ID'].unique()

In [97]:
pqbs

array(['2017_Matt Ryan', '2017_Tom Brady', '2017_Aaron Rodgers',
       '2017_Colin Kaepernick', '2017_Sam Bradford', '2017_Alex Smith',
       '2017_Ryan Fitzpatrick', '2017_Landry Jones', '2017_Andy Dalton',
       '2017_Matthew Stafford', '2017_Blake Bortles',
       '2017_Trevor Siemian', '2017_Russell Wilson', '2017_Carson Wentz',
       '2017_Matt Moore', '2017_Robert Griffin', '2017_Andrew Luck',
       '2017_Carson Palmer', '2017_Trevone Boykin', '2017_Drew Brees',
       '2017_Connor Cook', '2017_Eli Manning', '2017_Jared Goff',
       '2017_Brock Osweiler', '2017_Philip Rivers', '2017_Kirk Cousins',
       '2017_Matt Cassel', '2017_Jameis Winston', '2017_Joe Flacco',
       '2017_Tom Savage', '2017_Dak Prescott', '2017_Matt McGloin',
       '2017_Matt Barkley', '2017_EJ Manuel', '2017_David Fales',
       '2017_Cam Newton', '2017_Cardale Jones', '2017_Mark Sanchez',
       '2017_Sean Mannion', '2017_Derek Carr', '2017_Ben Roethlisberger',
       '2017_Tyrod Taylor', '2017_Mik

In [323]:
qbs_2017_14 = Week14_df.loc[Week14_df['QB_ID'].str.split('_', 1, expand=True)[0]=='2017']
qbs_2017_13 = Week13_df.loc[Week13_df['QB_ID'].str.split('_', 1, expand=True)[0]=='2017']
qbs_2017_12 = Week12_df.loc[Week12_df['QB_ID'].str.split('_', 1, expand=True)[0]=='2017']
qbs_2017_11 = Week11_df.loc[Week11_df['QB_ID'].str.split('_', 1, expand=True)[0]=='2017']

In [324]:
qbs14 = np.array(qbs_2017_14['QB_ID'])
qbs13 = np.array(qbs_2017_13['QB_ID'])
qbs12 = np.array(qbs_2017_12['QB_ID'])
qbs11 = np.array(qbs_2017_11['QB_ID'])

In [325]:
qbs_1merge = list(set(qbs14).intersection(qbs13))
qbs_2merge = list(set(qbs_1merge).intersection(qbs12))
qbs = list(set(qbs_2merge).intersection(qbs11))

In [296]:
qbs

['2017_Tom Savage',
 '2017_Carson Wentz',
 '2017_Tom Brady',
 '2017_Case Keenum',
 '2017_Brett Hundley',
 '2017_Blake Bortles',
 '2017_Alex Smith',
 '2017_Russell Wilson',
 '2017_Dak Prescott',
 '2017_Drew Brees',
 '2017_Marcus Mariota',
 '2017_DeShone Kizer',
 '2017_Joe Flacco',
 '2017_Andy Dalton',
 '2017_Derek Carr',
 '2017_Philip Rivers',
 '2017_Mitch Trubisky',
 '2017_Blaine Gabbert',
 '2017_Ben Roethlisberger',
 '2017_Matthew Stafford',
 '2017_Kirk Cousins',
 '2017_Jared Goff',
 '2017_Matt Ryan']

In [141]:
from statsmodels.tsa.arima_model import _arma_predict_out_of_sample

# get what you need for predicting one-step ahead
params = ar_yards.params
residuals = ar_yards.resid
p = ar_yards.k_ar
q = ar_yards.k_ma
k_exog = ar_yards.k_exog
k_trend = ar_yards.k_trend
steps = 1

oos_predictions_yards = _arma_predict_out_of_sample(params, steps, residuals, 
                                p, q, k_trend, k_exog, 
                                endog=residuals, exog=None, start=15)

In [142]:
oos_predictions_yards_resid

array([-1.015211])

In [423]:
qbs_working= ['2017_Tom Brady', '2017_Blake Bortles', 
              '2017_Alex Smith', '2017_Russell Wilson', '2017_Drew Brees', 
              '2017_Joe Flacco', '2017_Philip Rivers', 
              '2017_Matthew Stafford', '2017_Kirk Cousins', '2017_Jared Goff']

In [359]:
Fantasy_Points_Calculator('2017_Joe Flacco', 15)

array([ 8.7452356])

In [358]:
Fantasy_Points_Calculator('2017_Blake Bortles', 15)

array([ 10.88122469])

In [331]:
Fantasy_Points_Calculator('2017_Alex Smith', 15)

array([ 18.07242396])

In [332]:
Fantasy_Points_Calculator('2017_Russell Wilson', 15)

array([ 15.08177612])

In [353]:
Fantasy_Points_Calculator('2017_Matt Ryan', 15)

array([ 18.06802724])

In [ ]:
Fantasy_Points_Calculator('2017_Blake Bortles', 15)

In [ ]:
Fantasy_Points_Calculator('2017_Blake Bortles', 15)

In [ ]:
Fantasy_Points_Calculator('2017_Blake Bortles', 15)

In [ ]:
Fantasy_Points_Calculator('2017_Blake Bortles', 15)

In [424]:
week_15_projections = []
for i in qbs_working:
    projections = Fantasy_Points_Calculator(i, 15)[0]
    week_15_projections.append(projections)

In [425]:
proj_df = pd.DataFrame(week_15_projections)

In [426]:
proj_df['Projections'] = proj_df[0]

In [427]:
proj_df.drop([0], axis=1, inplace=True)

In [428]:
proj_df['QB'] = qbs_working

In [422]:
proj_df

,Projections,QB
0,20.953711,2017_Tom Brady
1,10.881225,2017_Blake Bortles
2,18.072424,2017_Alex Smith
3,15.081776,2017_Russell Wilson
4,16.215238,2017_Drew Brees
5,8.745236,2017_Joe Flacco
6,18.134847,2017_Philip Rivers
7,16.840742,2017_Matthew Stafford
8,15.741082,2017_Kirk Cousins
9,16.840403,2017_Jared Goff


In [372]:
test_data = pd.read_csv('Test data')

In [374]:
test_data['Rank_QBR'] = test_data['Rk']
test_data['Sack'] = test_data['Sk']
test_data['Completions'] = test_data['Cmp']
test_data['Attempts'] = test_data['Att']
test_data['Passing_Yards'] = test_data['Yds']
test_data['Sack_Yards'] = test_data['Yds.1']
test_data['QBR'] = test_data['Rate']
test_data['Yds_Per_Attempt'] =  test_data['Y/A']
test_data['Adj_Yd_Per_Attempt'] = test_data['AY/A']

In [375]:
test_data['Year'] = test_data['Date'].str.split('-', 1, expand=True)[0]
test_data['Year'] = test_data['Year'].astype(str).astype(int)
test_data['Month'] = test_data['Date'].str.split('-', 2, expand=True)[1]
test_data['Month'] = test_data['Month'].astype(str).astype(int)
test_data['Name'] = test_data['Unnamed: 1'].str.split('\\', 1, expand=True)[0]
test_data['Home_Away'] = (test_data['Unnamed: 6']=='@').map({False:'Home', True:'Away'})
test_data['Home_Team'] = (test_data['Home_Away']=='Home').map({True:1, False:0})
test_data['Fantasy_Points'] = (test_data['Passing_Yards']/25)+(data['TD']*6)-(data['Int']*2)

In [376]:
QB_ID = []
for a, b in zip(test_data['Year'], test_data['Name']):
    Qid = '{}_{}'.format(a,b)
    QB_ID.append(Qid)
    
test_data['QB_ID'] = QB_ID

In [407]:
test_data[["Unnamed: 1", 'Fantasy_Points']]

,Unnamed: 1,Fantasy_Points
0,Brock Osweiler\OsweBr00,49.76
1,Blake Bortles\BortBl00,31.04
2,Case Keenum\KeenCa00,21.44
3,Cam Newton\NewtCa00,15.68
4,Alex Smith\SmitAl03,19.24
5,Kirk Cousins\CousKi00,19.84
6,Nick Foles\FoleNi00,25.48
7,Matthew Stafford\StafMa00,21.48
8,Marcus Mariota\MariMa01,25.64
9,Ben Roethlisberger\RoetBe00,29.24


In [405]:
missing_qbs = set(test_data['QB_ID']) - set(proj_df['QB'])

In [ ]:
test_data

In [431]:
actual_fantasy_pts = [33.92, 31.04, 19.24, 15.68, 19.40, 21.52, 9.08, 21.48, 19.84, 14.80]


proj_df['actual_pts'] = actual_fantasy_pts

In [436]:
proj_df['se'] = (proj_df['Projections'] - proj_df['actual_pts'])**2

In [438]:
proj_df.drop('rmse',axis=1,inplace=True)

In [442]:
rmse = np.sqrt(proj_df['se'].mean())

In [443]:
rmse

9.3489926493359796

In [365]:
week_15_projections = []
for i in qbs:
    try:
        projections = Fantasy_Points_Calculator(i, 15)[0]
        week_15_projections.append(projections)
    except:
        print(i)
        break
    

2017_Tom Savage


In [257]:
data.loc[data['QB_ID']=='2017_Tom Savage']['Int']

2433    0
2705    0
2738    2
2752    1
2810    2
2838    1
2881    0
Name: Int, dtype: int64

In [51]:
Week_15_projections = Fantasy_Points_Calculator(QB_2017, 15)

NameError: name 'QB_2017' is not defined

In [220]:
yards = yard_projections(data, '2014_Peyton Manning', 2, lr_yards, ar_yards)
TD = TD_projections(data, '2014_Peyton Manning', 2, lr_TD, ar_TD)
Ints = Int_projections(data, '2014_Peyton Manning', 2, lr_Int, ar_Int)

['g', 'a', 'c']

In [128]:
data.loc[data['QB_ID']=='2014_Peyton Manning']

,Age,Date,Tm,Opp,Result,G#,Week,Day,Cmp%,TD,...,QBR,Yds_Per_Attempt,Adj_Yd_Per_Attempt,Year,Month,Name,Home_Away,Home_Team,QB_ID,Fantasy_Points
599,38-167,2014-09-07,DEN,IND,W 31-24,1,1,Sun,61.11,3,...,111.9,7.47,9.14,2014,9,Peyton Manning,Home,1,2014_Peyton Manning,28.76
634,38-230,2014-11-09,DEN,OAK,W 41-17,9,10,Sun,70.45,5,...,111.9,7.73,7.95,2014,11,Peyton Manning,Away,0,2014_Peyton Manning,39.60
685,38-237,2014-11-16,DEN,STL,L 7-22,10,11,Sun,62.96,1,...,75.3,7.20,5.91,2014,11,Peyton Manning,Away,0,2014_Peyton Manning,17.56
694,38-244,2014-11-23,DEN,MIA,W 39-36,11,12,Sun,80.00,4,...,135.4,7.34,9.63,2014,11,Peyton Manning,Home,1,2014_Peyton Manning,34.28
746,38-251,2014-11-30,DEN,KAN,W 29-16,12,13,Sun,50.00,2,...,85.3,5.26,6.44,2014,11,Peyton Manning,Away,0,2014_Peyton Manning,19.16
793,38-258,2014-12-07,DEN,BUF,W 24-17,13,14,Sun,70.00,0,...,56.9,8.65,4.15,2014,12,Peyton Manning,Home,1,2014_Peyton Manning,2.92
802,38-265,2014-12-14,DEN,SDG,W 22-10,14,15,Sun,70.00,1,...,125.6,11.65,12.65,2014,12,Peyton Manning,Away,0,2014_Peyton Manning,15.32
870,38-273,2014-12-22,DEN,CIN,L 28-37,15,16,Mon,63.64,2,...,61.8,7.07,3.89,2014,12,Peyton Manning,Away,0,2014_Peyton Manning,16.44
901,38-279,2014-12-28,DEN,OAK,W 47-14,16,17,Sun,56.76,0,...,80.1,7.38,7.38,2014,12,Peyton Manning,Home,1,2014_Peyton Manning,10.92
922,38-174,2014-09-14,DEN,KAN,W 24-17,2,2,Sun,80.77,3,...,143.9,9.31,11.62,2014,9,Peyton Manning,Home,1,2014_Peyton Manning,27.68


In [97]:
data['Passing_Yards'].loc[data['QB_ID']=='2014_Peyton Manning']

599     269
634     340
685     389
694     257
746     179
793     173
802     233
870     311
901     273
922     242
982     303
1041    479
1074    237
1102    318
1144    286
1194    438
Name: Passing_Yards, dtype: int64

In [74]:
data['Passing_Yards'][]

0    462
Name: Passing_Yards, dtype: int64

In [242]:
p_13_hat = lr_yards.predict(peyton_13)

In [243]:
p_13_hat

array([ 332.25375778,  321.20055572,  275.37970925,  205.20029177,
        320.42678748,  397.25766565,  297.78777995,  354.3627831 ,
        271.76038609,  315.26899881,  340.52211455,  330.59862009,
        383.98076174,  305.25320622,  338.63383161,  362.27231315])

In [252]:
peyton_13.head()

,Cmp%,TD,Int,Sack,Completions,Attempts,Sack_Yards,QBR
0,64.29,7,0,3,27,42,17,141.1
37,69.44,4,0,2,25,36,17,135.2
78,60.00,1,0,0,24,40,0,94.1
127,52.78,2,1,2,19,36,18,70.4
141,62.86,5,2,0,22,35,0,118.2


In [280]:
## Break Down By Week
Week1_df = data.loc[data['Week']==1]
Week2_df = data.loc[data['Week']==2]
Week3_df = data.loc[data['Week']==3]
Week4_df = data.loc[data['Week']==4]
Week5_df = data.loc[data['Week']==5]
Week6_df = data.loc[data['Week']==6]
Week7_df = data.loc[data['Week']==7]
Week8_df = data.loc[data['Week']==8]
Week9_df = data.loc[data['Week']==9]
Week10_df = data.loc[data['Week']==10]
Week11_df = data.loc[data['Week']==11]
Week12_df = data.loc[data['Week']==12]
Week13_df = data.loc[data['Week']==13]
Week14_df = data.loc[data['Week']==14]
Week15_df = data.loc[data['Week']==15]
Week16_df = data.loc[data['Week']==16]
Week17_df = data.loc[data['Week']==17]

## Break Down By Season
df_2017 = data.loc[data['Year']==2017]
df_2016 = data.loc[data['Year']==2016]
df_2015 = data.loc[data['Year']==2015]
df_2014 = data.loc[data['Year']==2014]
df_2013 = data.loc[data['Year']==2013]

average out of sample prediction streams